In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Cell 2
# import unsloth # Remove
import torch
# from unsloth import FastLanguageModel # Remove
import pickle # Not used in the provided snippet, can be removed if not used elsewhere
import os # Not used in the provided snippet, can be removed if not used elsewhere

from transformers import AutoModelForCausalLM, AutoTokenizer 
from peft import PeftModel # Add
print(f"PyTorch version: {torch.__version__}")

2025-06-05 08:27:19.507612: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749112039.715972      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749112039.782291      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


PyTorch version: 2.6.0+cu124


In [2]:
# Cell 4
adapter_repo_id = "aryan6637/deepseek-crop-fertilizer-info-v3"
base_model_name = "deepseek-ai/deepseek-llm-7b-base" # Base model used for fine-tuning
max_seq_length = 1024

In [3]:
# Cell 5
print(f"Loading base model {base_model_name} for CPU...")
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float32,  # Use float32 for CPU; bfloat16 might work on some newer CPUs
    # device_map="auto" will try to use GPU if available, then CPU.
    # Forcing CPU:
    # device_map="cpu", # Or handle with .to('cpu') later
)

print(f"Loading tokenizer from {adapter_repo_id}...")
tokenizer = AutoTokenizer.from_pretrained(adapter_repo_id, trust_remote_code=True) # Some tokenizers need this

print(f"Loading adapter from {adapter_repo_id} and merging...")
# Load the LoRA adapter
model = PeftModel.from_pretrained(model, adapter_repo_id)
# Merge the adapter into the base model for potentially faster CPU inference
# This increases memory usage as it creates a full model.
model = model.merge_and_unload()

model.to("cpu") # Explicitly move to CPU
print("Model and tokenizer loaded successfully on CPU.")
print("WARNING: Running a 7B parameter model on CPU will be very slow for inference.")


# Handle tokenizer padding - Unsloth logs indicate <|PAD_TOKEN|> is used.
if tokenizer.pad_token is None or tokenizer.pad_token_id is None:
    print("Tokenizer does not have a pad token or pad_token_id. Adding '<|PAD_TOKEN|>'.")
    # Check if the token already exists to avoid adding duplicates if config is weird
    if '<|PAD_TOKEN|>' not in tokenizer.get_vocab():
        tokenizer.add_special_tokens({'pad_token': '<|PAD_TOKEN|>'})
    else:
        tokenizer.pad_token = '<|PAD_TOKEN|>'
    # After adding or setting, ensure pad_token_id is correct
    tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

    # Resize model embeddings if new tokens were truly added (not just setting existing)
    # This is crucial if add_special_tokens actually modified the vocab size
    model.resize_token_embeddings(len(tokenizer))
    print(f"Resized model embeddings to: {len(tokenizer)}")


# Ensure the model's config pad_token_id is also updated to match the tokenizer
if model.config.pad_token_id is None or model.config.pad_token_id != tokenizer.pad_token_id:
    print(f"Updating model.config.pad_token_id to {tokenizer.pad_token_id}")
    model.config.pad_token_id = tokenizer.pad_token_id

print(f"Pad token: {tokenizer.pad_token}, ID: {tokenizer.pad_token_id}")
print(f"EOS token: {tokenizer.eos_token}, ID: {tokenizer.eos_token_id}")

Loading base model deepseek-ai/deepseek-llm-7b-base for CPU...


config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/355 [00:00<?, ?B/s]

Loading adapter from aryan6637/deepseek-crop-fertilizer-info-v3 and merging...


adapter_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/150M [00:00<?, ?B/s]

Model and tokenizer loaded successfully on CPU.
Updating model.config.pad_token_id to 100015
Pad token: <|PAD_TOKEN|>, ID: 100015
EOS token: <｜end▁of▁sentence｜>, ID: 100001


In [10]:
def format_inference_prompt(final_parameters):
    instruction_text = (
        f"Given the following soil and environmental parameters:\n"
        f"- Temperature: {final_parameters['Temparature']}°C\n"
        f"- Humidity: {final_parameters['Humidity']}%\n"
        f"- Moisture: {final_parameters['Moisture']}\n"
        f"- Soil Type: {final_parameters['Soil Type']}\n"
        f"- Nitrogen: {final_parameters['Nitrogen']} ppm\n"
        f"- Potassium: {final_parameters['Potassium']} ppm\n"
        f"- Phosphorous: {final_parameters['Phosphorous']} ppm\n\n"
        f"Predict the suitable Crop Type and Fertilizer Name, and provide brief information about how they work or their characteristics."
    )
    # Alpaca format - the model was trained to generate text after "### Response:\n"
    formatted_prompt = f"### Instruction:\n{instruction_text}\n\n### Response:\n"
    return formatted_prompt

In [11]:
# Cell 8
def get_prediction(final_parameters):
    inference_prompt = format_inference_prompt(final_parameters)
    # Tokenize the prompt
    device = "cpu" # Explicitly set to CPU
    inputs = tokenizer(inference_prompt, return_tensors="pt").to(device)

    print("\nGenerating response (this will be slow on CPU)...")
    with torch.no_grad(): # Ensure no gradients are calculated during inference
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id, # Use the configured pad_token_id
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )

    full_response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\n--- Full Generated Text (with prompt) ---")
    print(full_response_text)
    response_marker = "### Response:\n"
    prompt_length_in_tokens = inputs.input_ids.shape[1]
    generated_tokens = outputs[0][prompt_length_in_tokens:]
    response_only_text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    print("\n--- Model's Recommendation ---")
    print(response_only_text)

In [12]:
parameters ={'Temparature': 30.0,
 'Humidity': 45.0,
 'Moisture': 12.0,
 'Soil Type': 'red',
 'Nitrogen': 45,
 'Potassium': 3,
 'Phosphorous': 34}

In [14]:
prediction = get_prediction(parameters)


Generating response (this will be slow on CPU)...

--- Full Generated Text (with prompt) ---
### Instruction:
Given the following soil and environmental parameters:
- Temperature: 30.0°C
- Humidity: 45.0%
- Moisture: 12.0
- Soil Type: red
- Nitrogen: 45 ppm
- Potassium: 3 ppm
- Phosphorous: 34 ppm

Predict the suitable Crop Type and Fertilizer Name, and provide brief information about how they work or their characteristics.

### Response:
Recommended Crop Type: Oil seeds
Recommended Fertilizer: 10-26-26


--- Model's Recommendation ---
Recommended Crop Type: Oil seeds
Recommended Fertilizer: 10-26-26


In [ ]:
prediction